In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy.linalg
import scipy

print(np.__version__)
print(matplotlib.__version__)
print(scipy.__version__)

1.26.0
3.8.0
1.11.2


In [2]:
n = 99
#n = 4
a, b, c, p = 1, 10, 1, 1

A = np.matrix(np.zeros((n + 1, n + 1)))
i, j = np.indices(A.shape)
A[i==j + 1] = a
A[i==j] = b
A[i==j - 1] = c
A[n, j] = p

#A[i, j] = j + (n + 1) * i + 1

f = np.matrix(np.zeros((n + 1, 1)))
k, l = np.indices(f.shape)
f[k, 0] = k + 1

#f[0,0]
A

matrix([[10.,  1.,  0., ...,  0.,  0.,  0.],
        [ 1., 10.,  1., ...,  0.,  0.,  0.],
        [ 0.,  1., 10., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ..., 10.,  1.,  0.],
        [ 0.,  0.,  0., ...,  1., 10.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.]])

In [3]:
def normVector(v):
    return np.linalg.norm(v)

def __calcResidual(A, f, X):
    r = np.matrix(np.zeros((n + 1, 1)))
    r = A * X - f
    
    return r

def calcResidual(A, f, X):
    return normVector(__calcResidual(A, f, X))

In [4]:
def makePlot():
    plt.figure(figsize=(20, 10))

    plt.grid()
    plt.legend(fontsize=16)
    plt.show()

In [5]:
def solveGauss(A, f, n):
    A = np.matrix(A)
    f = np.matrix(f)
    #i, j = np.indices(A.shape)
    #k, l = np.indices(f.shape)
    #print(A.shape[0])

    for j in range(n):
        max_ind = j + np.argmax(np.abs(A[j:, j]))
        if max_ind != j:
            A[j, :], A[max_ind, :] = np.copy(A[max_ind, :]), np.copy(A[j, :])
            f[j, :], f[max_ind, :] = np.copy(f[max_ind, :]), np.copy(f[j, :])

        for i in range(j + 1, n + 1):
            frac = A[i, j] / A[j, j]
            #print(frac)
            A[i, :] -= A[j, :] * frac
            f[i, :] -= f[j, :] * frac
    
    if np.any(np.isclose(np.diag(A), 0.0)):
        raise Exception("Infinite number of solutions!")

    X = np.matrix(np.zeros((n + 1, 1)))
    for i in range(n, -1, -1):
        X[i, 0] = (f[i, 0] - A[i, i:] * X[i:, 0]) / A[i, i]

    return X

In [6]:
#with np.printoptions(precision=3, suppress=True):
    #solution = solveGauss(A, f, n)
    #print(solution)

solution = solveGauss(A, f, n)
calcResidual(A, f, solution)

/tmp/ipykernel_19400/1085303398.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  X[i, 0] = (f[i, 0] - A[i, i:] * X[i:, 0]) / A[i, i]


1.0883946155170226e-13

In [7]:
def __decomposeLU(A, n):
    P, L, U = scipy.linalg.lu(A)

    L = np.matrix(L)
    U = np.matrix(U)
    return np.matrix(L), np.matrix(U)

def solveLU(A, f, n):
    A = np.matrix(A)
    f = np.matrix(f)
    L, U = __decomposeLU(A, n)

    Y = np.matrix(np.zeros((n + 1, 1)))

    for i in range(n + 1):
        Y[i, 0] = f[i] - L[i, :i] * Y[:i]

    X = np.matrix(np.zeros((n + 1, 1)))

    for i in range(n, -1, -1):
        X[i, 0] = (Y[i] - U[i, i:] * X[i:, 0]) / U[i, i]

    return X
    

In [8]:
solution = solveLU(A, f, n)
calcResidual(A, f, solution)

/tmp/ipykernel_19400/776642697.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Y[i, 0] = f[i] - L[i, :i] * Y[:i]
/tmp/ipykernel_19400/776642697.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  X[i, 0] = (Y[i] - U[i, i:] * X[i:, 0]) / U[i, i]


7.868591628900475e-14

In [50]:
def solveSeidel(A, f, n, delta_th):
    A = np.matrix(A)
    f = np.matrix(f)
    
    X = np.matrix(np.zeros((n + 1, 1)))
    rs = list()

    while True:
        prev_X = np.matrix(X)

        for i in range(n + 1):
            X[i] = (f[i] - (A[i, :i] * X[:i] + A[i, i+1:] * X[i+1:])) / A[i, i]

        rs.append(calcResidual(A, f, X))
        if normVector(X - prev_X) < delta_th:
            break
    
    return X, np.array(rs)

In [51]:
solution, rs = solveSeidel(A, f, n, 1e-5)
calcResidual(A, f, solution)

4.078813365623585e-06